In [1]:
import pandas as pd
from nltk.util import ngrams
from nltk import trigrams
from ast import literal_eval
from collections import Counter, defaultdict

In [2]:
data = pd.read_csv("C:/Manning/DL/processedText.csv")

In [3]:
## Use this to convert list stored as string back to original list
data.loc[:,'token'] = data.loc[:,'token'].apply(lambda x: literal_eval(x))

In [4]:
train = data[data['category'] == 'title']
test = data[data['category'] != 'title']

In [6]:
# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

def trigram_c(text):
    for w1, w2, w3 in trigrams(text, pad_right=True, pad_left=True, left_pad_symbol='<s>', right_pad_symbol='</s>' ):
        model[(w1, w2)][w3] += 1

# update the model based on the tokens
data['token'].apply(trigram_c)

# Transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [19]:
dict(model[('priors','from')])

{'experts': 0.15789473684210525,
 'same': 0.05263157894736842,
 'the': 0.3157894736842105,
 'non': 0.05263157894736842,
 'data': 0.10526315789473684,
 'samples': 0.05263157894736842,
 'experimental': 0.05263157894736842,
 'my': 0.05263157894736842,
 'whom': 0.05263157894736842,
 'multiple': 0.05263157894736842,
 'b': 0.05263157894736842}

In [200]:
def generate_text(first, second):

    import random

    # starting words
    text = [first, second]
    sentence_finished = False

    while not sentence_finished:
      # select a random probability threshold  
      r = random.random()
      accumulator = 0.1

      for word in model[tuple(text[-2:])].keys():
          accumulator += model[tuple(text[-2:])][word]
          # select words that are above the probability threshold
          if accumulator >= r:
              text.append(word)
              if text[-1:] == ['</s>']:
                  sentence_finished = True
              break

    print(str(' '.join([t for t in text if t])))

In [202]:
generate_text('how','do')


how do i interpret the images are not identifiable , you may be better if you add some more technical terms what causes the latter , we have , from where you try to clarify in what you re getting there . what does discriminative mean here ? </s>


In [174]:
def sentence_prob(sentence):
    prob = 1
    for w1, w2, w3 in ngrams(sentence.split(), 3):
        prob *= model[(w1, w2)][w3]
        
    print(prob)
    

In [204]:
sentence_prob('that is great')

0.0005712109672505713


In [195]:
sentence_prob(' '.join([t for t in train['token'][10]]))


3.8131226710161687e-06


In [196]:
train['token'][10]

['bayesian', 'and', 'frequentist', 'reasoning', 'in', 'plain', 'english']

In [ ]:
P1 <- 1/4
P2 <- 1/4
P3 <- 1/4
P4 <- 1/120000

SP <- P1*P2*P3*P4
Result <- SP**(-1/4)